# Integração ETL, ELT, APIs e Arquivos

Você trabalha em um e-commerce de variados produtos, e no último mês os gerentes operacionais da companhia notaram que
houve uma variação muito grande em seus estoques por períodos de até 24h, cujos produtos, voltavam ao estoque 
no dia seguinte o que não caracterizava uma venda concretizada.
Sabendo disso um dos diretores sugeriu um estudo sobre os carrinhos em aberto, e qual o perfil dos usuários que deixavam seus carrinhos em aberto, e para isso fora criada uma demanda para seu time.

Para resolver esta demanda, você e seu time deverão realizar a ingestão dos dados de:

- Usuários
- Carrinhos 

Estes dados poderão ser obtidos através dos seguintes endpoints:

- Usuários: https://dummyjson.com/auth/users
- Carrinhos: https://dummyjson.com/auth/carts

Lembrando que estes são endpoints seguros então você deverá realizar a autenticação através do seguinte endpoint: https://dummyjson.com/auth/login

Lembre-se de verificar a documentação para obter mais detalhes do funcionamento dos endpoints https://dummyjson.com/docs/auth

Após isso, os passos de tratamento do dado, serão:

1. Após consumidos os dados, você deverá persistí-los utilizando a técnica aprendida em aula em uma pastas chamada raw no formato json

2. Consumir sua camada raw e criar uma nova entidade que relacione os clientes aos carrihos através do atributo userId do seu carrinho e o atribute id da tabela users, feito isso gravar o resultado dessa agregação em uma pasta chamda kitchen

3. Por fim você deverá contemplar se os dados gerados são capazes de responder as seguintes perguntas:

    - Qual o estado de residencia possui clientes com mais carrinhos em aberto ?
    - Qual a idade dos usuários com mais carrinhos em aberto ?
    - Quais os produtos que mais aparecem em carrinhos em aberto ?

In [21]:
# Importando bibliotecas
import pandas as pd 
import requests
import json
import os
from pandas import json_normalize

In [22]:
# Função para realizar a autenticação na API
def authenticate(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "username": login,
            "password": senha,
            "expiresInMins": 1
        })
    )
    
    return login_response.json()['token']

In [23]:
# Função para obter os cabeçalhos de autorização necessários para fazer solicitações autenticadas na API
def get_authorization_headers():
    access_token = authenticate(login="kminchelle", senha="0lelplR")
    
    authorization_header = {
        "Authorization": f"Bearer {access_token}", 
        "Content-Type": "application/json"
    }
    
    return authorization_header

In [24]:
# Função para fazer uma solicitação GET a uma URL especificada e retornar os dados JSON da resposta
def get_endpoint(url: str):
    response = requests.get(
        url=url, 
        headers=get_authorization_headers()
    )
    
    return response.json()
  
users = get_endpoint("https://dummyjson.com/auth/users")
carts = get_endpoint("https://dummyjson.com/auth/carts")

1. Após consumidos os dados, você deverá persistí-los utilizando a técnica aprendida em aula em uma pastas chamada raw no formato json

In [25]:
# Função para receber os dados (em formato JSON) e salvar os dados dentro da pasta "raw"
def save_data_to_file(data, file_name):
    with open(f"raw/{file_name}.json", mode="w") as file_to_write:
        json.dump(data, file_to_write)
        print(f"Dados de {file_name} foram salvos na pasta 'raw'.")

save_data_to_file(users, "users")
save_data_to_file(carts, "carts")

Dados de users foram salvos na pasta 'raw'.
Dados de carts foram salvos na pasta 'raw'.


2. Consumir sua camada raw e criar uma nova entidade que relacione os clientes aos carrihos através do atributo userId do seu carrinho e o atribute id da tabela users, feito isso gravar o resultado dessa agregação em uma pasta chamda kitchen

In [26]:
# Função para criar a nova entidade relacionando clientes aos carrinhos
def create_customer_cart_entity(users_data, carts_data):
    customer_carts = {}

    for _, cart in carts_data.iterrows():
        user_id = cart['userId']
        user_info = users_data[users_data['id'] == user_id].iloc[0].to_dict()

        if user_id in customer_carts:
            customer_carts[user_id]["carts"].append(cart.to_dict())
        else:
            customer_carts[user_id] = {"user_info": user_info, "carts": [cart.to_dict()]}

    return customer_carts

In [27]:
# Convertendo os dados brutos para dataframes
users_df = pd.DataFrame(users)
carts_df = pd.DataFrame(carts)

In [28]:
# Usando json_normalize para desmembrar o JSON na coluna 'carts'
normalized_carts_df = json_normalize(carts_df['carts'])
normalized_carts_df.head()

,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"[{'id': 59, 'title': 'Spring and summershoes',...",2328,1941,97,5,10
1,2,"[{'id': 96, 'title': 'lighting ceiling kitchen...",3023,2625,30,5,10
2,3,"[{'id': 37, 'title': 'ank Tops for Womens/Girl...",460,403,63,5,10
3,4,"[{'id': 36, 'title': 'Sleeve Shirt Womens', 'p...",553,493,83,5,10
4,5,"[{'id': 23, 'title': 'Orange Essence Food Flav...",844,745,58,5,10


In [29]:
# Fazendo o mesmo para 'users'
normalized_users_df = json_normalize(users_df['users'])
normalized_users_df.head()

,id,firstName,lastName,maidenName,age,gender,email,phone,username,password,...,bank.iban,company.address.address,company.address.city,company.address.coordinates.lat,company.address.coordinates.lng,company.address.postalCode,company.address.state,company.department,company.name,company.title
0,1,Terry,Medhurst,Smitham,50,male,atuny0@sohu.com,+63 791 675 8914,atuny0,9uQFF1Lh,...,NO17 0695 2754 967,629 Debbie Drive,Nashville,36.208114,-86.586212,37076,TN,Marketing,Blanda-O'Keefe,Help Desk Operator
1,2,Sheldon,Quigley,Cole,28,male,hbingley1@plala.or.jp,+7 813 117 7139,hbingley1,CQutx25i8r,...,MD63 L6YC 8YH4 QVQB XHIK MTML,8821 West Myrtle Avenue,Glendale,33.540430,-112.248839,85305,AZ,Services,Aufderhar-Cronin,Senior Cost Accountant
2,3,Terrill,Hills,Hoeger,38,male,rshawe2@51.la,+63 739 292 7942,rshawe2,OWsTbMUgFc,...,AT24 1095 9625 1434 9703,18 Densmore Drive,Essex,44.492953,-73.101883,05452,VT,Marketing,Lindgren LLC,Mechanical Systems Engineer
3,4,Miles,Cummerata,Maggio,49,male,yraigatt3@nature.com,+86 461 145 4186,yraigatt3,sRQxjPfdS,...,KZ43 658B M6VS TZOU OXSO,210 Green Road,Manchester,41.790910,-72.511951,06042,CT,Business Development,Wolff and Sons,Paralegal
4,5,Mavis,Schultz,Yundt,38,male,kmeus4@upenn.edu,+372 285 771 1911,kmeus4,aUTdmmmbH,...,IT41 T114 5127 716J RGYB ZRUX DSJ,8398 West Denton Lane,Glendale,33.515353,-112.240812,85305,AZ,Support,Adams Inc,Web Developer I


In [30]:
# Realizando o merge dos dataframes
merged_df = normalized_users_df.merge(normalized_carts_df, on='id')
merged_df.head()

,id,firstName,lastName,maidenName,age,gender,email,phone,username,password,...,company.address.state,company.department,company.name,company.title,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,Terry,Medhurst,Smitham,50,male,atuny0@sohu.com,+63 791 675 8914,atuny0,9uQFF1Lh,...,TN,Marketing,Blanda-O'Keefe,Help Desk Operator,"[{'id': 59, 'title': 'Spring and summershoes',...",2328,1941,97,5,10
1,2,Sheldon,Quigley,Cole,28,male,hbingley1@plala.or.jp,+7 813 117 7139,hbingley1,CQutx25i8r,...,AZ,Services,Aufderhar-Cronin,Senior Cost Accountant,"[{'id': 96, 'title': 'lighting ceiling kitchen...",3023,2625,30,5,10
2,3,Terrill,Hills,Hoeger,38,male,rshawe2@51.la,+63 739 292 7942,rshawe2,OWsTbMUgFc,...,VT,Marketing,Lindgren LLC,Mechanical Systems Engineer,"[{'id': 37, 'title': 'ank Tops for Womens/Girl...",460,403,63,5,10
3,4,Miles,Cummerata,Maggio,49,male,yraigatt3@nature.com,+86 461 145 4186,yraigatt3,sRQxjPfdS,...,CT,Business Development,Wolff and Sons,Paralegal,"[{'id': 36, 'title': 'Sleeve Shirt Womens', 'p...",553,493,83,5,10
4,5,Mavis,Schultz,Yundt,38,male,kmeus4@upenn.edu,+372 285 771 1911,kmeus4,aUTdmmmbH,...,AZ,Support,Adams Inc,Web Developer I,"[{'id': 23, 'title': 'Orange Essence Food Flav...",844,745,58,5,10


In [31]:
# Verificando as colunas do dataframe
merged_df.columns

Index(['id', 'firstName', 'lastName', 'maidenName', 'age', 'gender', 'email',
       'phone', 'username', 'password', 'birthDate', 'image', 'bloodGroup',
       'height', 'weight', 'eyeColor', 'domain', 'ip', 'macAddress',
       'university', 'ein', 'ssn', 'userAgent', 'hair.color', 'hair.type',
       'address.address', 'address.city', 'address.coordinates.lat',
       'address.coordinates.lng', 'address.postalCode', 'address.state',
       'bank.cardExpire', 'bank.cardNumber', 'bank.cardType', 'bank.currency',
       'bank.iban', 'company.address.address', 'company.address.city',
       'company.address.coordinates.lat', 'company.address.coordinates.lng',
       'company.address.postalCode', 'company.address.state',
       'company.department', 'company.name', 'company.title', 'products',
       'total', 'discountedTotal', 'userId', 'totalProducts', 'totalQuantity'],
      dtype='object')

In [32]:
# Especificando o caminho completo para a pasta "kitchen" e o nome do arquivo CSV
caminho_kitchen = r'D:\Downloads\- cursos\Santander Coders\05_EXTRAÇÃO_DADOS_I\Extração de Dados I - Aula 3 - 20 de set\kitchen'
nome_arquivo = 'shopping.csv'

# Verificando se o diretório "kitchen" existe, e criando caso não exista
if not os.path.exists(caminho_kitchen):
    os.makedirs(caminho_kitchen)

# Salvando o dataframe em um arquivo CSV na pasta "kitchen"
merged_df.to_csv(os.path.join(caminho_kitchen, nome_arquivo), index=False)

3. Por fim você deverá contemplar se os dados gerados são capazes de responder as seguintes perguntas:

    - Qual o estado de residencia possui clientes com mais carrinhos em aberto ?

In [33]:
state_counts = merged_df['address.state'].value_counts().reset_index()
state_counts.columns = ['estado', 'carrinhos']
state_counts.head()

,estado,carrinhos
0,KY,3
1,CA,3
2,TN,3
3,CT,2
4,VT,2


    - Qual a idade dos usuários com mais carrinhos em aberto ?

In [34]:
age_counts = merged_df['age'].value_counts().reset_index()
age_counts.columns = ['idade', 'carrinhos']
age_counts.head()

,idade,carrinhos
0,38,2
1,21,2
2,42,2
3,26,2
4,41,2


    - Quais os produtos que mais aparecem em carrinhos em aberto ?

In [35]:
most_common_products = merged_df['products'].explode().apply(lambda x: x['title']).value_counts()
most_common_products_df = most_common_products.reset_index()
most_common_products_df.columns = ['produto', 'carrinhos']
most_common_products_df.head()

,produto,carrinhos
0,Gulab Powder 50 Gram,3
1,printed high quality T shirts,3
2,Leather Strap Skeleton Watch,3
3,Pubg Printed Graphic T-Shirt,3
4,Spring and summershoes,2
